In [21]:
'''The goal is to build all blocks all possible blocks in our seach space so that it can be run on fpga and
latency profiling can be done to build the latency table'''

import time
import copy
import torch
import torch.nn as nn
import numpy as np
from ofa.utils.layers import *
from ofa.utils.pytorch_modules import make_divisible
import ast

In [22]:
base_resolution = 224
batch_size = 1
multiplier = 1.2

block, input_size, in_channels, out_channels, expand_ratio, kernel_size, stride, act, se

In [23]:
configurations = [
    # (ConvLayer, base_resolution, 3, 16, 3, 2, "relu"),
    # (
    #     ResidualBlock,
    #     base_resolution // 2,
    #     16,
    #     16,
    #     [1],
    #     [3],
    #     1,
    #     "relu",
    #     False,
    # ),
    # (
    #     ResidualBlock,
    #     base_resolution // 2,
    #     16,
    #     24,
    #     [6],
    #     [3, 5, 7],
    #     2,
    #     "relu",
    #     False,
    # ),
    (
        ResidualBlock,
        base_resolution // 4,
        24,
        24,
        [6],
        [3, 5, 7],
        1,
        "relu",
        False,
    ),
    (
        ResidualBlock,
        base_resolution // 4,
        24,
        24,
        [6],
        [3, 5, 7],
        1,
        "relu",
        False,
    ),
    (
        ResidualBlock,
        base_resolution // 4,
        24,
        24,
        [6],
        [3, 5, 7],
        1,
        "relu",
        False,
    ),
    (
        ResidualBlock,
        base_resolution // 4,
        24,
        40,
        [6],
        [3, 5, 7],
        2,
        "relu",
        True,
    ),
    (
        ResidualBlock,
        base_resolution // 8,
        40,
        40,
        [6],
        [3, 5, 7],
        1,
        "relu",
        True,
    ),
    (
        ResidualBlock,
        base_resolution // 8,
        40,
        40,
        [6],
        [3, 5, 7],
        1,
        "relu",
        True,
    ),
    (
        ResidualBlock,
        base_resolution // 8,
        40,
        40,
        [6],
        [3, 5, 7],
        1,
        "relu",
        True,
    ),
    (
        ResidualBlock,
        base_resolution // 8,
        40,
        80,
        [6],
        [3, 5, 7],
        2,
        "h_swish",
        False,
    ),
    (
        ResidualBlock,
        base_resolution // 16,
        80,
        80,
        [6],
        [3, 5, 7],
        1,
        "h_swish",
        False,
    ),
    (
        ResidualBlock,
        base_resolution // 16,
        80,
        80,
        [6],
        [3, 5, 7],
        1,
        "h_swish",
        False,
    ),
    (
        ResidualBlock,
        base_resolution // 16,
        80,
        80,
        [6],
        [3, 5, 7],
        1,
        "h_swish",
        False,
    ),
    (
        ResidualBlock,
        base_resolution // 16,
        80,
        112,
        [4],
        [3, 5, 7],
        1,
        "h_swish",
        True,
    ),
    (
        ResidualBlock,
        base_resolution // 16,
        112,
        112,
        [4],
        [3, 5, 7],
        1,
        "h_swish",
        True,
    ),
    (
        ResidualBlock,
        base_resolution // 16,
        112,
        112,
        [4],
        [3, 5, 7],
        1,
        "h_swish",
        True,
    ),
    (
        ResidualBlock,
        base_resolution // 16,
        112,
        112,
        [4],
        [3, 5, 7],
        1,
        "h_swish",
        True,
    ),
    (
        ResidualBlock,
        base_resolution // 16,
        112,
        160,
        [4],
        [3, 5, 7],
        2,
        "h_swish",
        True,
    ),

    (
        ResidualBlock,
        base_resolution // 32,
        160,
        160,
        [4],
        [3, 5, 7],
        1,
        "h_swish",
        True,
    ),
    (
        ResidualBlock,
        base_resolution // 32,
        160,
        160,
        [4],
        [3, 5, 7],
        1,
        "h_swish",
        True,
    ),
    (
        ResidualBlock,
        base_resolution // 32,
        160,
        160,
        [4],
        [3, 5, 7],
        1,
        "h_swish",
        True,
    )
    # (ConvLayer, base_resolution // 32, 160, 960, 1, 1, "h_swish"),
    # (ConvLayer, 1, 960, 1280, 1, 1, "h_swish"),
    # (LinearLayer, 1, 1280, 1000, 1, 1)
]

In [24]:
def is_equal(block1, block2):
    # Function to check if two blocks are equal
    return block1[0] == block2[0] and block1[1:] == block2[1:]

unique_configurations = []

for config in configurations:
    # Check if the current configuration is already in the unique_configurations list
    if not any(is_equal(config, unique_config) for unique_config in unique_configurations):
        unique_configurations.append(config)
        
len(unique_configurations)

9

In [36]:
# retruns unique name for a block
def build_name(cfg, input_shape):
    name = []
    name.append(cfg['name'])
    cfg = cfg['mobile_inverted_conv']
    in_name = 'x'.join(map(str, input_shape))
    name.append(f'in_{in_name}')
    out = cfg['out_channels']
    name.append(f'out_{out}')
    k = cfg['kernel_size']
    name.append(f'k_{k}')
    s = cfg['stride']
    name.append(f's_{s}')
    act = cfg['act_func']
    name.append(f'act_{act}')
    use_se = cfg['use_se']
    name.append(f'use_se_{use_se}')
    
    name = '-'.join(name)
    return name


def build_layers(configurations=configurations):
#Dictionary to store layer, input_shape
    Layers = [] # {"layer" : "input",}
    i = 0
    for layer_idx in range(len(configurations)):
        config = configurations[layer_idx]
        op_type = config[0]
        if op_type == ResidualBlock:
            (
                _,
                input_size,
                in_channels,
                out_channels,
                expand_list,
                ks_list,
                stride,
                act,
                se,
            ) = config
            in_channels = make_divisible(int(round(in_channels * multiplier)), 8)
            out_channels = make_divisible(int(round(out_channels * multiplier)), 8)
            template_config = {
                "name": ResidualBlock.__name__,
                "mobile_inverted_conv": {
                    "name": MBConvLayer.__name__,
                    "in_channels": in_channels,
                    "out_channels": out_channels,
                    "kernel_size": 0,
                    "stride": stride,
                    "expand_ratio": 0,
                    # 'mid_channels': None,
                    "act_func": act,
                    "use_se": se,
                },
                "shortcut": {
                    "name": IdentityLayer.__name__,
                    "in_channels": in_channels,
                    "out_channels": out_channels,
                }
                if (in_channels == out_channels and stride == 1)
                else None,
            }
            sub_dict = {}
            for ks in ks_list:
                for e in expand_list:
                    build_config = copy.deepcopy(template_config)
                    build_config["mobile_inverted_conv"]["expand_ratio"] = e
                    build_config["mobile_inverted_conv"]["kernel_size"] = ks
                    layer = ResidualBlock.build_from_config(build_config)
                    input_shape = [in_channels, input_size, input_size]
                    name = build_name(build_config, input_shape)
                    input_shape_batch = [batch_size, in_channels, input_size, input_size] 
                    print(name)
                    Layers.append([name, layer, input_shape_batch])
                        
        elif op_type == ConvLayer:
            (
                _,
                input_size,
                in_channels,
                out_channels,
                kernel_size,
                stride,
                activation,
            ) = config
            if in_channels > 8:
                in_channels = make_divisible(int(round(in_channels * multiplier)), 8)
                
            out_channels = make_divisible(int(round(out_channels * multiplier)), 8)
            build_config = {
                # 'name': ConvLayer.__name__,
                "in_channels": in_channels,
                "out_channels": out_channels,
                "kernel_size": kernel_size,
                "stride": stride,
                "dilation": 1,
                "groups": 1,
                "bias": False,
                "use_bn": True,
                "has_shuffle": False,
                "act_func": activation,
            }
            layer = ConvLayer.build_from_config(build_config)
            input_shape = (batch_size, in_channels, input_size, input_size)
            Layers[layer] = input_shape

        elif op_type == LinearLayer:
            _, input_size, in_channels, out_channels, kernel_size, stride = config
            in_channels = make_divisible(int(round(in_channels * multiplier)), 8)
            out_channels = out_channels
            build_config = {
                # 'name': LinearLayer.__name__,
                "in_features": in_channels,
                "out_features": out_channels,
            }
            layer = LinearLayer.build_from_config(build_config)
            input_shape = (batch_size, in_channels)
            Layers[layer] = input_shape
        else:
            raise NotImplementedError
    return Layers

In [13]:
# # funntion to group together layers that have same input shape
# def group_by_values(dictionary):
#     grouped_dict = {}
#     for key, value in dictionary.items():
#         if value in grouped_dict:
#             grouped_dict[value].append(key)
#         else:
#             grouped_dict[value] = [key]
#     return grouped_dict

# layers_grouped_by_input = group_by_values(all_layers)  #dict: {'in_shape' : [layer1, layer2], ...}

In [37]:
all_layers = build_layers(configurations=unique_configurations)

ResidualBlock-in_32x56x56-out_32-k_3-s_1-act_relu-use_se_False
ResidualBlock-in_32x56x56-out_32-k_5-s_1-act_relu-use_se_False
ResidualBlock-in_32x56x56-out_32-k_7-s_1-act_relu-use_se_False
ResidualBlock-in_32x56x56-out_48-k_3-s_2-act_relu-use_se_True
ResidualBlock-in_32x56x56-out_48-k_5-s_2-act_relu-use_se_True
ResidualBlock-in_32x56x56-out_48-k_7-s_2-act_relu-use_se_True
ResidualBlock-in_48x28x28-out_48-k_3-s_1-act_relu-use_se_True
ResidualBlock-in_48x28x28-out_48-k_5-s_1-act_relu-use_se_True
ResidualBlock-in_48x28x28-out_48-k_7-s_1-act_relu-use_se_True
ResidualBlock-in_48x28x28-out_96-k_3-s_2-act_h_swish-use_se_False
ResidualBlock-in_48x28x28-out_96-k_5-s_2-act_h_swish-use_se_False
ResidualBlock-in_48x28x28-out_96-k_7-s_2-act_h_swish-use_se_False
ResidualBlock-in_96x14x14-out_96-k_3-s_1-act_h_swish-use_se_False
ResidualBlock-in_96x14x14-out_96-k_5-s_1-act_h_swish-use_se_False
ResidualBlock-in_96x14x14-out_96-k_7-s_1-act_h_swish-use_se_False
ResidualBlock-in_96x14x14-out_136-k_3-s_1-a

### need to deploy these models 

In [39]:
for layer in all_layers:
    name = layer[0]
    input = layer[2]
    model = layer[1]
    model.eval()
    try: 
        o = model(torch.rand(input))
    except:
        print('Failed to inference layer{}'.format(name))
        break
    
    file_path = f'all_blocks/{name}.pth'
    torch.save(model, file_path)
print('Done')

Done


In [38]:
!mkdir all_blocks

### Defining the Model 
no code utilized below this

In [48]:
# Takes all the blocks that have same shape and places them in parallel
class myModel(nn.Module):
    def __init__(self, layers, input):
        super(myModel, self).__init__()
        self.first = nn.Conv2d(in_channels= input[1], out_channels= input[1], kernel_size= 1)
        self.blocks = nn.ModuleList(layers)

    def forward(self, x):
        output = []
        x = self.first(x)
        for i, _ in enumerate(self.blocks):
            y = self.blocks[i](x)
            output.append(y)
        return output
    
# get shape of all the output tensor from model 
def get_ouptut_shaped(output):
    shape = []
    for value in output:
        s = value.shape
        shape.append(tuple(s))
    return shape

In [49]:
# # Build a big model by combining differnt models according to the input output shape match

# #Creating a single model combining all the layers that have same input shape

# models = []  # [model_archi, input_shape, output_shapes]

# for key in iter(layers_grouped_by_input):
#     if len(key)==4:
#         model = myModel(layers_grouped_by_input[key], key)
#         model.eval()
#         input = torch.rand(key)
#         output = model(input)
#         out_shape = get_ouptut_shaped(output)

#         models.append([model, key, out_shape])  # All models in the shape [arch, input, output_list]


In [50]:
# capture the output(list) from previous model, go through output list, find the matching output to the input to next layer
prev_outputs = 0
for i, arch in enumerate(models):
    current_input = arch[1]
    if prev_outputs != 0:
        try:
            index = prev_outputs.index(current_input)
        except ValueError:
            index = None
            
        models[i].append(index)
    prev_outputs = arch[2]
    
#models-> [arch, input, output_list, index], index capture which output brach to connect from list of all previous outputs 


In [52]:
!mkdir blocks

In [53]:
for i, model in enumerate(models):
    s = None
    o= None
    m = model[0]
    inp = model[1]
    # s = summary(m, (inp[1], inp[2], inp[3]))
    o = m(torch.randn(inp))
    # print(o[0])
    if o is not None:
        t = [str(inp[1]), str(inp[2]), str(inp[3])]
        s = '_'.join(t)
        print(s)
        torch.save(m, f'blocks/b{i}_{s}.pth')
        print(f'block_{i}_saved')

3_224_224
block_0_saved
24_112_112
block_1_saved
32_56_56
block_2_saved
48_28_28
block_3_saved
96_14_14
block_4_saved
136_14_14
block_5_saved
192_7_7
block_6_saved
1152_1_1
block_7_saved


In [17]:
# Build a larger model if possible by combaning the models based on input output shape
class MyLargeModel(nn.Module):
    def __init__(self, model1, model2, index):
        super(MyLargeModel, self).__init__()
        self.block1 = model1
        self.block2 = model2
        self.index = index

    def forward(self, x):
        y1 = self.block1(x)
        y = self.block2(y1[self.index])  # as models reutun list as defined
        return y

In [18]:

#Takes list of smaller models and returns a combined model 
def combine_models(models):
    big_model = 0
    
    for i, model in enumerate(models):   
        if big_model !=0: 
            arch = model[0]
            index = model[3]
            if index is not None:
                big_model = MyLargeModel(big_model, arch, index)
            else:
                print(f"Cannot continue combaning modesl, Indices: {i}" )
                break
            
        else:
            big_model = model[0]
            input = model[1]
            
    return big_model, input
    

In [19]:
from torchsummary import summary

In [20]:
big, input = combine_models(models[2:3])
torch.save(big, 'b23_s32_56_56.pth')
summary(big, (32, 56, 56))
big(torch.randn(input))
input

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 56, 56]           1,056
            Conv2d-2           [-1, 96, 56, 56]           3,072
       BatchNorm2d-3           [-1, 96, 56, 56]             192
              ReLU-4           [-1, 96, 56, 56]               0
            Conv2d-5           [-1, 96, 56, 56]             864
       BatchNorm2d-6           [-1, 96, 56, 56]             192
              ReLU-7           [-1, 96, 56, 56]               0
            Conv2d-8           [-1, 32, 56, 56]           3,072
       BatchNorm2d-9           [-1, 32, 56, 56]              64
      MBConvLayer-10           [-1, 32, 56, 56]               0
    IdentityLayer-11           [-1, 32, 56, 56]               0
    ResidualBlock-12           [-1, 32, 56, 56]               0
           Conv2d-13          [-1, 128, 56, 56]           4,096
      BatchNorm2d-14          [-1, 128,

(1, 32, 56, 56)

In [21]:
print(big)

myModel(
  (first): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  (blocks): ModuleList(
    (0): ResidualBlock(
      (conv): MBConvLayer(
        (inverted_bottleneck): Sequential(
          (conv): Conv2d(32, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (depth_conv): Sequential(
          (conv): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=96, bias=False)
          (bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (point_linear): Sequential(
          (conv): Conv2d(96, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (shortcut): IdentityLayer()
    )
    (1): ResidualBlock(
      (conv):

In [22]:
big, input = combine_models(models[4:6])
torch.save(big, f'b45_96_14_14.pth')
summary(big, (96, 14, 14) )
big(torch.randn(input))
input

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 14, 14]           9,312
            Conv2d-2          [-1, 288, 14, 14]          27,648
       BatchNorm2d-3          [-1, 288, 14, 14]             576
            Hswish-4          [-1, 288, 14, 14]               0
            Conv2d-5          [-1, 288, 14, 14]           2,592
       BatchNorm2d-6          [-1, 288, 14, 14]             576
            Hswish-7          [-1, 288, 14, 14]               0
            Conv2d-8           [-1, 96, 14, 14]          27,648
       BatchNorm2d-9           [-1, 96, 14, 14]             192
      MBConvLayer-10           [-1, 96, 14, 14]               0
    IdentityLayer-11           [-1, 96, 14, 14]               0
    ResidualBlock-12           [-1, 96, 14, 14]               0
           Conv2d-13          [-1, 384, 14, 14]          36,864
      BatchNorm2d-14          [-1, 384,

(1, 96, 14, 14)

In [27]:
import torch.onnx 

#Function to Convert to ONNX 
def Convert_ONNX(model, in_shape, name): 

    # set the model to inference mode 
    model.eval() 

    # Let's create a dummy input tensor  
    dummy_input = torch.randn(in_shape, requires_grad=True)  

    # Export the model   
    torch.onnx.export(model,         # model being run 
         dummy_input,       # model input (or a tuple for multiple inputs) 
         f"{name}.onnx",       # where to save the model  
         export_params=True,  # store the trained parameter weights inside the model file 
         opset_version=10,    # the ONNX version to export the model to 
         do_constant_folding=True,  # whether to execute constant folding for optimization 
         input_names = ['modelInput'],   # the model's input names 
         output_names = ['modelOutput'], # the model's output names 
         dynamic_axes={'modelInput' : {0 : 'batch_size'},    # variable length axes 
                                'modelOutput' : {0 : 'batch_size'}}) 
    print(" ") 
    print('Model has been converted to ONNX')